In [1]:
import shapely.geometry
import requests
import netCDF4
import tempfile
import json
import time

from ipywidgets import *
from ipyleaflet import *

In [2]:
with open('domains.json') as f:
    data = json.load(f)

peace_coords = [
    (coord[0], coord[1]) for coord in data['borders']['peace']['coordinates']
]
fraser_coords = [
    (coord[0], coord[1]) for coord in data['borders']['fraser']['coordinates']
]
columbia_coords = [
    (coord[0], coord[1]) for coord in data['borders']['columbia']['coordinates']
]

In [3]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat = round(kwargs.get('coordinates')[0], 5)
        lon = round(kwargs.get('coordinates')[1], 5)

        polygon = in_polygon(lat, lon)
        
        if polygon and point_list.options:
            if (in_polygon(float(point_list.options[0].split(', ')[0]), float(point_list.options[0].split(', ')[1]))) == polygon:
                    
                clear_marker()

                latitude.value = f'{lat}'
                longitude.value = f'{lon}'

                m.add_layer(Marker(location=kwargs.get('coordinates'), name='Marker', draggable=False))
                
        elif polygon and not point_list.options:
                
            clear_marker()

            latitude.value = f'{lat}'
            longitude.value = f'{lon}'

            m.add_layer(Marker(location=kwargs.get('coordinates'), name='Marker', draggable=False))

output_widget = Output()
@output_widget.capture()
def handle_run(arg):
    valid = True
    if not point_list.options:
        print('Please add at least one point before continuing')
        valid = False
    if not start_date.value and not end_date.value:
        print('Please enter a start and end date before continuing')
        valid = False
    if valid:
        with output_widget: # Output print statements to main workflow instead of logs
            #Start RVIC process
            url = build_url(start_date.value, end_date.value, points)
            print(url)
            input_response = requests.get(f'http://docker-dev03.pcic.uvic.ca:30111/osprey/input?{url}').content
            print(input_response.decode('utf-8'))
            status_url = input_response.split()[-1].decode('utf-8')            

            #Check status of RVIC process
            status_response = requests.get(f'http://docker-dev03.pcic.uvic.ca:30111{status_url}').content
            print(status_response.decode('utf-8'))

            while 'Process is still running.' in status_response.decode('utf-8'):
                time.sleep(5)
                status_response = requests.get(f'http://docker-dev03.pcic.uvic.ca:30111{status_url}').content
                print(status_response.decode('utf-8'))
            
            print()
            output_url = status_response.split()[-1].decode('utf-8')
            outputs.append(output_url)
            

        
def handle_add(arg):
    if f'{latitude.value}, {longitude.value}' not in points:
        points.append(f'{latitude.value}, {longitude.value}')
        point_list.options = points
    
    new_icon = AwesomeIcon(name="map-pin", marker_color='orange')
    new_marker = Marker(location = (latitude.value, longitude.value), icon=new_icon, draggable=False)
    
    if new_marker not in marker_group.layers:
        clear_marker()
        marker_group.add_layer(new_marker)
    
    
def handle_remove(arg):
    if point_list.value:
        for point in point_list.value:
            points.remove(point)
            
        for marker in marker_group.layers:
            if ', '.join(marker.location) in point_list.value:
                marker_group.remove_layer(marker)
            
    point_list.options = points

In [4]:
def clear_marker():
    for i in range(1, len(m.layers)):
        if m.layers[i].name == 'Marker':
            m.remove_layer(m.layers[i])


def in_polygon(lat, lon):

    point = shapely.geometry.Point(lat, lon)

    peace_polygon = shapely.geometry.polygon.Polygon(p.locations)
    fraser_polygon = shapely.geometry.polygon.Polygon(f.locations)
    columbia_polygon = shapely.geometry.polygon.Polygon(c.locations)

    if (peace_polygon.contains(point)):
        return ('Peace')
    
    elif (fraser_polygon.contains(point)):
        return ('Fraser')
        
    elif (columbia_polygon.contains(point)):
        return ('Columbia')

    else:
        return None


def date_widget(descr):
    return DatePicker(description=descr, disabled=False)


def build_url(start, end, points):
   
    start_val = '&run_startdate=' + str(start) + '-00'
    end_val = '&stop_date=' + str(end)
    
    lon_val = '&lons=' + ','.join([point.split(', ')[1] for point in points])
    lat_val = '&lats=' + ','.join([point.split(', ')[0] for point in points])

    url = 'case_id=sample' + start_val + end_val + lon_val + lat_val

    return url

In [5]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = 'Default'

satellite = basemap_to_tiles(basemaps.Gaode.Satellite)
satellite.base = True
satellite.name = 'Satellite'

m = Map(
    basemap=mapnik,
    center=(50.5, -120),
    zoom=5,
    layout=Layout(height='700px'),
    layers=[satellite, mapnik],
)

p = Polygon(
    locations=peace_coords,
    color='blue',
    name='Peace',
)
f = Polygon(
    locations=fraser_coords,
    color='red',
    name='Fraser',
)
c = Polygon(
    locations=columbia_coords,
    color='green',
    name='Columbia',
)

legend = LegendControl(
    {'Peace': 'blue', 'Fraser': 'red', 'Columbia': 'green'},
    name='Watersheds',
    position='topright',
)

latitude = Text(
    placeholder='Enter the Latitude', description='Latitude:', disabled=False
)

longitude = Text(
    placeholder='Enter the Longitude', description='Longitude:', disabled=False
)

layer_control = LayersControl()
marker_group = LayerGroup()

point_list = SelectMultiple(options=[], value=[], description='Points:', disabled=False)
points = []

outputs = []
run = Button(
    description='Run',
    button_style='success',
    disabled=False,
    tooltip="Click 'Run' to start the Osprey Flask App.",
)
run.on_click(handle_run)

add_point = Button(
    description='Add Point',
    button_style='primary',
    disabled=False,
    tooltip='Click to add this point to the list.',
)
add_point.on_click(handle_add)

remove_point = Button(
    description='Remove Point(s)',
    button_style='danger',
    disabled=False,
    tooltip='Click to remove selected point(s) from the list.',
)
remove_point.on_click(handle_remove)

start_date = date_widget('Start Date:')
end_date = date_widget('End Date:')

In [6]:
m.add_layer(p)
m.add_layer(f)
m.add_layer(c)
m.add_layer(marker_group)

m += layer_control

m.on_interaction(handle_click)
m.add_control(legend)

In [7]:
box_layout = Layout(display='flex',
                flex_flow = 'column', 
                width='110%',
                align_items = 'center')

control_box = Box(children = [start_date, end_date, latitude, longitude, add_point, point_list, remove_point, run], layout=box_layout)
display(AppLayout(center = m, 
    right_sidebar = control_box,
    align_items = 'center'))
display(output_widget)

AppLayout(children=(Box(children=(DatePicker(value=None, description='Start Date:'), DatePicker(value=None, de…

Output()

In [8]:
outputs

['/osprey/output/543b931c-3658-4b77-b7dc-793c8122a47f',
 '/osprey/output/34e7530a-93f7-4313-aacd-199cf114d5a3',
 '/osprey/output/1dc8cbbc-27c5-40d4-b7d7-cf0d86ab271c']

In [9]:
output = requests.get(f'http://docker-dev03.pcic.uvic.ca:30111{outputs[-1]}').content

In [10]:
#Saves data as NetCDF data set
with tempfile.NamedTemporaryFile(dir=os.getenv('TMPDIR', default='/tmp/')) as fp:
    fp.write(output)
    data = netCDF4.Dataset(fp.name) 

In [11]:
#Inspect the structure of the output data
print(data.dimensions.items())

dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 9), ('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2), ('outlets', <class 'netCDF4._netCDF4.Dimension'>: name = 'outlets', size = 2), ('nc_chars', <class 'netCDF4._netCDF4.Dimension'>: name = 'nc_chars', size = 256)])


In [12]:
#To download file run this cell
watershed = in_polygon(float(points[0].split(',')[0]), float(points[0].split(',')[1]))
file_name = watershed.lower() + '.rvic.h0a.' + str(end_date.value) + '.nc'
home = os.path.expanduser('~')

print('The file has been downloaded here', os.path.join(home, 'Downloads', file_name))

with open(os.path.join(home, 'Downloads', file_name), 'wb') as file:
    file.write(output) 

The file has been downloaded here /home/eyvorchuk/Downloads/peace.rvic.h0a.2023-10-10.nc
